In [2]:
import pyvista as pv
import numpy as np
from pathlib import Path
import pygmsh
import meshio

In [3]:
avatar_path = Path("../").resolve() / "data" / "avatar" / "spread_arms.obj"
avatar_mesh = pv.read(avatar_path)

avatar_mesh = avatar_mesh.scale([0.01, 0.01, 0.01])

/Users/vikramsingh/Library/Caches/pypoetry/virtualenvs/pbd-GYzgUBp8-py3.8/lib/python3.8/site-packages/pyvista/core/pointset.py:262: PyVistaDeprecationWarning: You did not specify a value for `inplace` and the default value will be changing to `False` in future versions for point-based meshes (e.g., `PolyData`). Please make sure you are not assuming this to be an inplace operation.
  warnings.warn(DEFAULT_INPLACE_WARNING, PyVistaDeprecationWarning)


In [4]:
lower_right_points = [[0, 0.95, 0.25], [0.25, 0.95, 0.25], [0.25, 1.25, 0.25]]
right_sleeve_points = [[0.35, 1.20, 0.25], [0.35, 1.39,0.25] ]
center_points = [[0.15, 1.55,0.25], [-0.15, 1.55,0.25],]
left_sleeve_points = [[-0.35, 1.39,0.25], [-0.35, 1.20, 0.25]]
# lower_left_points = [[-0.25, 1.25, 0.25], [-0.25, 0.95, 0.25], [0., 0.95, 0.25]]
lower_left_points = [[-0.25, 1.25, 0.25], [-0.25, 0.95, 0.25]]

tshirt_points = lower_right_points + right_sleeve_points + center_points + left_sleeve_points + lower_left_points
tshirt_lines = pv.lines_from_points(tshirt_points)

In [10]:
def create_mesh(closed_lines: np.ndarray) -> pv.PolyData:
    with pygmsh.geo.Geometry() as geom:
        geom.add_polygon(
            closed_lines,
            mesh_size=0.1,
        )

        mesh: meshio.Mesh = geom.generate_mesh(
            dim=2, algorithm=5, verbose=True
        )  # 5: Delaunay

    # HardCoding alert! We observe that [1] has triangle elements so we use that
    # https://github.com/meshpro/pygmsh/discussions/524
    elems = mesh.cells[1].data
    coords = mesh.points

    cells = [("triangle", elems)]
    meshio.write_points_cells("/tmp/out.vtk", coords, cells)

    mesh_pv = pv.read("/tmp/out.vtk")

    return mesh_pv

triangle_mesh_front = create_mesh(tshirt_lines.points)
triangle_mesh_back = triangle_mesh_front.copy(deep=True)
triangle_mesh_back.translate((0, 0, -0.5), inplace=True)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 70%] Meshing curve 8 (Line)
Info    : [ 80%] Meshing curve 9 (Line)
Info    : [ 90%] Meshing curve 10 (Line)
Info    : [100%] Meshing curve 11 (Line)
Info    : Done meshing 1D (Wall 0.000660862s, CPU 0.001051s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Delaunay)
Info    : Done meshing 2D (Wall 0.00258022s, CPU 0.002561s)
Info    : 99 nodes 207 elements


UnstructuredGrid,Information
N Cells,167
N Points,99
X Bounds,"-3.500e-01, 3.500e-01"
Y Bounds,"9.500e-01, 1.550e+00"
Z Bounds,"-2.500e-01, -2.500e-01"
N Arrays,0


In [16]:
center = (0, 1.1, 0.25)

plane = pv.Plane(center=center, direction=(0, 0, 1), i_size=0.5, j_size=0.3)

plotter = pv.Plotter()
plotter.add_mesh(avatar_mesh, color="yellow")
# plotter.add_mesh(plane, color="blue")
plotter.add_mesh(triangle_mesh_back, color = "blue", show_edges=True)
plotter.add_mesh(triangle_mesh_front, color = "blue", show_edges=True)

edge_points = triangle_mesh_front.extract_feature_edges(boundary_edges=True, 
                                      non_manifold_edges=False, 
                                      manifold_edges=False)

plotter.add_point_labels(edge_points.points, labels = [str(point) for point in edge_points.points])

plotter.add_axes()
plotter.show(cpos = 'xy')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [28]:
edge_point_indexes = [triangle_mesh_front.find_closest_point(point) for point in edge_points.points] 

print(edge_point_indexes)
for index in edge_point_indexes[5:]:
    print(index, triangle_mesh_front.points[index])

[14, 2, 9, 25, 6, 21, 5, 19, 0, 11, 12, 16, 4, 7, 23, 17, 18, 20, 28, 22, 26, 10, 1, 13, 27, 24, 15, 3, 8]
21 [-0.21666667  1.49666667  0.25      ]
5 [0.15 1.55 0.25]
19 [0.05 1.55 0.25]
0 [0.   0.95 0.25]
11 [0.08333333 0.95       0.25      ]
12 [0.16666667 0.95       0.25      ]
16 [0.35  1.295 0.25 ]
4 [0.35 1.39 0.25]
7 [-0.35  1.39  0.25]
23 [-0.35   1.295  0.25 ]
17 [0.28333333 1.44333333 0.25      ]
18 [0.21666667 1.49666667 0.25      ]
20 [-0.05  1.55  0.25]
28 [-0.08333333  0.95        0.25      ]
22 [-0.28333333  1.44333333  0.25      ]
26 [-0.25  1.05  0.25]
10 [-0.25  0.95  0.25]
1 [0.25 0.95 0.25]
13 [0.25 1.05 0.25]
27 [-0.16666667  0.95        0.25      ]
24 [-0.3    1.225  0.25 ]
15 [0.3   1.225 0.25 ]
3 [0.35 1.2  0.25]
8 [-0.35  1.2   0.25]


In [23]:
triangle_mesh_front_file = Path("../").resolve() / "data" / "avatar" / "tshirt_front.obj"
triangle_mesh_back_file = Path("../").resolve() / "data" / "avatar" / "tshirt_back.obj"
pv.save_meshio(filename=triangle_mesh_front_file, mesh=triangle_mesh_front)
pv.save_meshio(filename=triangle_mesh_back_file, mesh=triangle_mesh_back)